In [1]:
import pyspark
from pyspark.sql import Column, DataFrame, SparkSession, functions
from pyspark.sql.functions import *
from py4j.java_collections import MapConverter
import shutil
import random
import threading
conf = pyspark.SparkConf()
conf.setMaster("spark://spark:7077") 

conf.set("spark.hadoop.fs.s3a.endpoint", 'http://s3:9000') \
    .set("spark.hadoop.fs.s3a.access.key", 'minio') \
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
sc = pyspark.SparkContext(conf=conf)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.databricks#dbutils-api_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-50b67d9b-a630-407e-a471-77378a516813;1.0
	confs: [default]
	found com.databricks#dbutils-api_2.12;0.0.5 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.

In [2]:
!pip install koalas==1.8.1

     |████████████████████████████████| 1.4 MB 955 kB/s            


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession(sc).builder.appName("streaming").getOrCreate()
from delta.tables import *

In [4]:
import time
OBJECTURL_TEST = 's3a://minio-sink-bucket/topics/customers'

# Read the data back from MinIO
gnames_df = spark.read.format('json') \
    .load(OBJECTURL_TEST)
gnames_df.show()

21/10/21 16:20:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------+------+---+--------------------+-----------+-------------+---------+
|               after|before| op|              source|transaction|        ts_ms|partition|
+--------------------+------+---+--------------------+-----------+-------------+---------+
|{sally.thomas@acm...|  null|  r|{mysql, inventory...|       null|1634832232389|        0|
|{annek@noanswer.o...|  null|  r|{mysql, inventory...|       null|1634832232389|        0|
|{gbailey@foobar.c...|  null|  r|{mysql, inventory...|       null|1634832232389|        0|
|{ed@walker.com, E...|  null|  r|{mysql, inventory...|       null|1634832232389|        0|
+--------------------+------+---+--------------------+-----------+-------------+---------+



In [5]:
from pyspark.sql.functions import from_json, col
gnames_df.select(col("after.id"), col("after.first_name"), col("after.last_name"), col("after.email")).collect()

[Row(id=1001, first_name='Sally', last_name='Thomas', email='sally.thomas@acme.com'),
 Row(id=1004, first_name='Anne', last_name='Kretchmar', email='annek@noanswer.org'),
 Row(id=1002, first_name='George', last_name='Bailey', email='gbailey@foobar.com'),
 Row(id=1003, first_name='Edward', last_name='Walker', email='ed@walker.com')]

In [36]:
#getting the schema from existing file
schema=spark.read.format('json').load(OBJECTURL_TEST).schema
spark.readStream.format("json").schema(schema).load(OBJECTURL_TEST).writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", 's3a://minio-sink-bucket/delta/customer/_checkpoints/streaming-agg').start('s3a://minio-sink-bucket/bronze/customers')

In [30]:
spark.readStream.format("json").schema(schema).load(OBJECTURL_TEST)

DataFrame[after: struct<email:string,first_name:string,id:bigint,last_name:string>, before: string, op: string, source: struct<connector:string,db:string,file:string,gtid:string,name:string,pos:bigint,query:string,row:bigint,server_id:bigint,snapshot:string,table:string,thread:string,ts_ms:bigint,version:string>, transaction: string, ts_ms: bigint, partition: int]

In [6]:
from databricks import koalas as ks
kdf = gnames_df.to_koalas()

In [7]:
kdf['after'][0]

21/10/21 16:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Row(email='sally.thomas@acme.com', first_name='Sally', id=1001, last_name='Thomas')

In [ ]:
table_dir = "s3a://test-container/playground/delta-table"
spark.sql("CREATE TABLE delta.`%s`(id LONG) USING delta" % table_dir)


In [ ]:
spark.sql("VACUUM '%s' RETAIN 169 HOURS" % table_dir).collect()

In [ ]:
import random

data = spark.range(8)
data = data.withColumn("value", data.id + random.randint(0, 5000))
data.take(10)

In [ ]:
spark.sql("INSERT INTO delta.`%s` VALUES 0, 1, 2, 3, 4" % table_dir)

In [ ]:
spark.sql("SELECT * FROM delta.`%s`" % table_dir).show()

In [ ]:
!pip install koalas==1.8.1

In [ ]:
import time
OBJECTURL_TEST = 's3a://test-container/playground/colors-test' + str(time.time()) + '.csv'
rdd = sc.parallelize([('Mario', 'Red'), ('Luigi', 'Green'), ('Princess', 'Pink')])
rdd.toDF(['name', 'color']).write.csv(OBJECTURL_TEST, header=True)

# Read the data back from MinIO
gnames_df = spark.read.format('csv').option('header', True) \
    .load(OBJECTURL_TEST)
gnames_df.show()

In [ ]:
DELTA_URL='s3a://test-container/playground/delts-colors-test' + str(time.time())
gnames_df.write.format("delta").save(DELTA_URL)

In [ ]:
data = spark.range(0, 5)
#data.write.format("delta").save("/tmp/delta-table")

In [ ]:
data.write.format("delta").save("s3a://test-container/playground/to_overide")

In [ ]:
data.write.format("delta").mode("overwrite").save("s3a://test-container/playground/to_overide")

In [ ]:
from databricks import koalas as ks
import pandas as pd
import numpy as np

In [ ]:
s = ks.Series([1, 3, 5, np.nan, 6, 8])

In [ ]:
kdf = ks.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [ ]:
kdf

In [ ]:
dates = pd.date_range('20130101', periods=6)
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
kdf = ks.from_pandas(pdf)
print(type(kdf))
kdf

In [ ]:
sdf = spark.createDataFrame(pdf)
sdf.show()

In [ ]:
kdf = sdf.to_koalas()

In [ ]:
sdf.collect()

In [ ]:
kdf.to_parquet("s3a://test-container/playground/kdf")

In [ ]:
ks.read_parquet("s3a://test-container/playground/kdf").head(10)

In [ ]:
kdf.to_delta("s3a://test-container/playground/delta_partitioned", mode='overwrite', partition_cols=["A","B"])

In [ ]:
ks.read_delta("s3a://test-container/playground/delta").head(10)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable
import shutil
import random

In [ ]:
data = spark.range(8)
data = data.withColumn("value", data.id + random.randint(0, 5000))
data.write.format("delta").mode("overwrite").save("s3a://test-container/playground/delta-streaming/delta-table")

In [ ]:
# Stream writes to the table
print("####### Streaming write ######")
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream\
    .format("delta")\
    .option("checkpointLocation", "s3a://test-container/playground/delta-streaming/checkpoint")\
    .start("s3a://test-container/playground/delta-streaming/delta-table2")
stream.awaitTermination(10)
stream.stop()

In [ ]:
spark.sql("SELECT * FROM delta.`%s`" % "s3a://test-container/playground/delta-streaming/delta-table2").show()

In [ ]:
# Stream reads from a table
print("##### Reading from stream ######")
stream2 = spark.readStream.format("delta").load("s3a://test-container/playground/delta-streaming/delta-table2")\
    .writeStream\
    .format("console")\
    .start()
stream2.awaitTermination(10)
stream2.stop()

In [ ]:
print("####### Streaming upgrades in update mode ########")

In [ ]:
# Function to upsert microBatchOutputDF into Delta Lake table using merge
def upsertToDelta(microBatchOutputDF, batchId):
    t = deltaTable.alias("t").merge(microBatchOutputDF.alias("s"), "s.id = t.id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

In [ ]:
from pyspark.sql.functions import *


streamingAggregatesDF = spark.readStream.format("rate").load()\
    .withColumn("id", col("value") % 10)\
    .drop("timestamp")

# Write the output of a streaming aggregation query into Delta Lake table
deltaTable = DeltaTable.forPath(spark, "s3a://test-container/playground/delta-streaming/delta-table")
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()


In [ ]:
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")

In [ ]:
streamingAggregatesDF = spark.readStream.format("rate").load()\
    .withColumn("id", col("value") % 10)\
    .drop("timestamp")
# Write the output of a streaming aggregation query into Delta Lake table
deltaTable = DeltaTable.forPath(spark, "s3a://test-container/playground/delta-streaming/delta-table")
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")
deltaTable.toDF().show()

# Streaming append and concurrent repartition using  data change = false
# tbl1 is the sink and tbl2 is the source
print("############ Streaming appends with concurrent table repartition  ##########")
tbl1 = "s3a://test-container/playground/delta-streaming/delta-table4"
tbl2 = "s3a://test-container/playground/delta-streaming/delta-table5"
numRows = 10
spark.range(numRows).write.mode("overwrite").format("delta").save(tbl1)
spark.read.format("delta").load(tbl1).show()
spark.range(numRows, numRows * 10).write.mode("overwrite").format("delta").save(tbl2)


# Start reading tbl2 as a stream and do a streaming write to tbl1
# Prior to Delta 0.5.0 this would throw StreamingQueryException: Detected a data update in the
# source table. This is currently not supported.
stream4 = spark.readStream.format("delta").load(tbl2).writeStream.format("delta")\
    .option("checkpointLocation", "s3a://test-container/playground/delta-streaming/checkpoint/tbl1") \
    .outputMode("append") \
    .start(tbl1)

# repartition table while streaming job is running
spark.read.format("delta").load(tbl2).repartition(10).write\
    .format("delta")\
    .mode("overwrite")\
    .option("dataChange", "false")\
    .save(tbl2)

stream4.awaitTermination(10)
stream4.stop()
print("######### After streaming write #########")
spark.read.format("delta").load(tbl1).show()
# cleanup
try:
    shutil.rmtree("s3a://test-container/playground/delta-streaming/")
except:
    pass

In [ ]:
spark.sql("CREATE TABLE delta.`%s`(id LONG)" % "s3a://test-container/playground/delta-table")

In [15]:
import DBUtils

ModuleNotFoundError: No module named 'DBUtils'

In [16]:
!conda install -c gwerbin databricks-utils

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - ^C
failed with repodata from current_repodata.json, will retry with next repodata source.

CondaError: KeyboardInterrupt



In [17]:
!pip install databricks-utils


  Preparing metadata (setup.py) ... done
  Created wheel for databricks-utils: filename=databricks_utils-0.0.7-py3-none-any.whl size=5251 sha256=a3364bb600cee0c44994bfed0e614778a671e8d7822d196baac9e3854bd48b75
  Stored in directory: /home/jovyan/.cache/pip/wheels/98/7e/30/1a3abd6df7bcc291066f0b8be7790406a9b885f6c613e357eb
Successfully built databricks-utils


In [19]:
import json
from databricks_utils.aws import S3Bucket
S3Bucket.attach_dbutils(dbutils)

NameError: name 'dbutils' is not defined

In [21]:
bucket = (S3Bucket("somebucketname", "SOMEACCESSKEY", "SOMESECRETKEY")
          .allow_spark(sc) # local spark context
          .mount("somebucketname"))

RuntimeError: `dbutils` not provided. Please call `S3Bucket.attach_dbutils` or provide `dbutils` in the arguments.